In [ ]:
# ! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastai.vision.all import *
from fastbook import *

# Stochastic Gradient Descent (SGD)

In our Pixel Similarity approach, we do not have any kind of weight assignment, or any way of improving based on testing the effectiveness based on a weight assignment. In otherwords, we can't really improve our Pixel Similiarity approach by modifying a set of parameters. In order to take advantage of deep learning, we first need to represent our task in the way that Arthur Samuel described it.

Instead of trying to find the similarity between an image and an "ideal image", we could instead look at each individual pixel and come up with a set of weights for each one, such that the highest weights are associated with those pixels most likely to be black for a particular category.
For instance, pixels towards the bottom right are not very likely to be activated for a 7, so they should have a low weight for a 7, but they are likely to be activated for an 8, so they should have a high weight for an 8.

This can be represented as a function and a set of weight values for each possible category.

In [ ]:
# Probability of being the number 8
def pr_eight(x,w): return (x*w).sum()

# x is the image, represented as a vector
# w is the vector of weights

# If we have the above function, then we just need some way to update the weights to make them a little bit better.
# With such an approach, we can repeat that step a number of times, making the weights better and better.

We want to find the specific values for the vector *w* that causes the result of our function to be high for those images that are actually 8s, and low for those images that are not. 
Searching for the best vector *w* is a way to search for the best function for recognising 8s.

To be more specific, here are the steps that we are going to require, to turn this function into a machine learning classifier:
1. Initialise the weights.
2. For each image, use these weights to predict whether it appears to be a 3 or a 7.
3. Based on these predictions, calculate how good the model is (its loss).
4. Calculate the gradient, which measures for each weight, how changing that weight would change the loss.
5. Step (that is, change) all the weights based on that calculation.
6. Go back to step 2 and repeat the process.
7. Iterate until you decide to stop the training process.

## Calculating Gradients

The derivative of a function tells us how much a change in its parameters will change its result.
If we know how our function wil lchange, then we know what we need to do to make it smaller.

This is the key to machine learning: having a way to change the parameters of a function to make it smaller.
Calculus provides us with a computational shortcut, the derivative, which lets us directly calculate the gradients of our functions.

One important thing to remember is that our function has lots of weights that we need to adjust, so when we calculate the derivative we won't get back one number, but lots of them - a gradient for every weight.
However, we can calculate the derivative with respect to one weight, and treat all the others as constant, then repeat that for each other weight.
This is how all of the gradients are calculated for every weight.

Fortunately, PyTorch is able to automatically compute the derivative of nearly any function very quickly.

In [ ]:
# A simple loss function
def f(x): return x**2

# A tensor value which we want gradients at
xt = tensor(3.).requires_grad_()

# requires_grad_() is a special method we use to tell PyTorch that we want to calculate gradients with respect to that variable at that value.
# It tells PyTorch to remember to keep track of how to compute gradients of the other, direct calculations on it that you will ask for.

# Calculating a function with that value.
yt = f(xt)
yt

In [ ]:
# Telling PyTorch to calculate the gradients for us
yt.backward()

# The "backward" here referes to backpropagation, which is the name given to the process of calculating the derivative of each layer.

In [ ]:
# Viewing the gradients attribute of our tensor
xt.grad

# The derivative of x**2 is 2*x
# Since we have x=3, the gradient should be 2*3=6 which is what PyTorch calculated for us

In [ ]:
# Repeating the above steps but with a vector argument for our function

def f(x): return (x**2).sum()       # Adding .sum() so it can take a rank-1 tensor and return a scalar

xt = tensor([3., 4., 10.]).requires_grad_()

yt = f(xt)

yt.backward()
xt.grad

The gradients only tell us the slope of our function, they don't actually tell us exactly how far to adjust the parameters. But it gives us some idea of how far; if the slope is very large, then that may suggest that we have more adjustments to do, whereas if the slope is very small, that may suggest that we are close to the optimal value.

## Stepping with Learning Rate

Deciding how to change our parameters based on the values of the gradients is an important part of the deep learning process.
Nearly all approaches start with the basic idea of multiplying the gradient by some small number, called the *learning rate* (LR).

The learning rate is often a number between 0.001 and 0.1, although it could be anything. Once you've picked a learning rate, you can adjust your parameters using the following simple function.


In [ ]:
# Optimiser Step
w -= gradient(w) * lr

This is known as *stepping* your parameters, using an *optimizer* step. 

Notice how we subtract the ```gradient * lr``` from the parameter to update it. This allows us to adjust the parameter in the direction of the slope by increasing the parameter when the slope is negative and decreasing the parameter when the slope is positive.

We want to adjust the parameters in the direction of the slope because our goal in deep learning is to *minimise* the loss.

## End-to-End SGD Example

Let's look at an SGD example and see how finding a minimum can be used to train a model to fit data better.

Starting with a simple, synthetic, example model. Imagine if you were measuring the speed of a roller coaster as it went over the top of a hump. It would start fast, and then get slower as it went up the hill; it would be slowest at the top, and it would then speed up again as it went downhill. You want to build a model of how the speed changes over time. If you were measuring the speed manually every second for 20 seconds, it might look something like this.

In [ ]:
# Time between 0 and 20s
time = torch.arange(0,20).float(); time

In [ ]:
# Speed of rollercoaster with respect to time
speed = torch.randn(20)*3 + 0.75*(time-9.5)**2 + 1
plt.scatter(time,speed)

Random noise was added, since measuring things manually isn't precise. This means its not that easy to answer the question: what was the roller coaster's speed?

Using SGD, we can try to find a function that matches our observations. We can't consider every possible function, so let's use a guess that it will be quadratic ```a*(time**2)+(b*time)+c```.

We want to distinguish clearly between the function's input (time) and its parameters. So let's collect the parameters in one argument and thus separate the input, *t*, and the parameters, *params* in the function's signature.

In [ ]:
def f(t, params):
    a,b,c = params
    return a*(t**2) + (b*t) + c

We have now restricted the problem of finding the best imaginable function that fits the data, to finding the best quadratic function.
This greatly simplifies the problem, since every quadratic function is fully defined by the three parameters *a, b and c*. Thus, to find the best quadratic function, we only need to find the best values for *a, b and c*.

We need to find first what we mean by "best". We define this precisely by choosing a *loss* function, which will return a value based on a prediction and a target, where lower values of the function correspond to "better" predictions. It is important for loss functions to return *lower* values when predictions are more accurate, as the SGD procedure we defined earlier will try to *minimise* this loss.

For continuous data, it's common to use *mean squared error*:

In [ ]:
# Sample Loss Function
def mse(preds, targets): return ((preds-targets)**2).mean()

Now let's work through Arthur Samuel's 7 Step Process.

### Step 1: Initialise the Parameters

First, we initialise the parameters to random values, and tell PyTorch that we want to track their gradients using ```requires_grad_```.

In [ ]:
# Initialising random parameters
params = torch.randn(3).requires_grad_()

### Step 2: Calculate Predictions

In [ ]:
# Calculating the predictions
preds = f(time, params)

Let's create a function to see how close our predictions are to our targets, and take a look.

In [ ]:
def show_preds(preds, ax=None):
    if ax is None: ax=plt.subplots()[1]
    ax.scatter(time, speed)
    ax.scatter(time, to_np(preds), color='red')
    ax.set_ylim(-300,100)

show_preds(preds)

This doesn't look very close. Our random parameters suggest that the rollercoaster will end up going backwards, since we have negative speeds.

### Step 3: Calculate the Loss

In [ ]:
# Calculating the loss using the mean squared error function we defined previously
loss = mse(preds, speed)
loss

Our goal is to improve this. To do that, we'll need to know the gradients.

### Step 4: Calculate the Gradients




In [ ]:
# Calculating an approximation of how the parameters need to change (aka. the gradient)

loss.backward()
params.grad

We can use these gradients to improve our parameters. We'll need to pick a learning rate which we'll use 1e-5 for now.

In [ ]:
params.grad * 1e-5

### Step 5: Step the Weights

We need to update the parameters based on the gradients we just calculated.

In [ ]:
# Optimiser Step
lr = 1e-5
params.data -= lr * params.grad.data
params.grad = None

Let's see if the loss has improved and take a look at the plot.

In [ ]:
preds = f(time,params)
mse(preds,speed)

In [ ]:
show_preds(preds)

It has reduced the error!
We now need to repeat this step a few times, so we'll create a function to apply one step.

In [ ]:
def apply_step(params, prn=True):
    preds = f(time, params)
    loss = mse(preds, speed)
    loss.backward()
    params.data -= lr * params.grad.data
    params.grad = None
    if prn: print(loss.item())
    return preds

### Step 6: Repeating the Process

We now iterate by looping and performing many improvements with the hope that we reach a good result.

In [ ]:
# Iterating over 10 epochs
for i in range(10): apply_step(params)

The loss is going down, just as we had hopped.

But looking only at these loss numbers diguises the fact that each iteration represents an entirely different quadratic function being tried, on the way to finding the best possible quadratic function.

We can see this process visually if, instead of printing out the loss function, we plot the function at every step.

In [ ]:
_,axs = plt.subplots(1,4,figsize=(12,3))
for ax in axs: show_preds(apply_step(params, False), ax)
plt.tight_layout()

### Step 7: Stop

We just decided to stop after 10 epochs arbitrarily. In practice, we would watch the training and validation losses and our metrics to decide when to stop.